In [2]:
import glob
import subprocess
import calendar

def gen_qsub(radar_id='', radar_start='', radar_end='', ncpu=16, mem=32, walltime=10, py_script='',queue_mode=''):
    #build header
    conf_txt = \
    """#!/bin/bash
    #PBS -P en0
    #PBS -q {qmode}
    #PBS -l walltime={time}:00:00
    #PBS -l mem={mem}GB
    #PBS -l wd
    #PBS -l ncpus={cpu}
    #PBS -l jobfs=150GB
    #PBS -m a
    #PBS -l storage=gdata/rq0+gdata/kl02+gdata/en0+gdata/lb4+gdata/rt52+gdata/wr45+scratch/kl02
    export PATH="/g/data/en0/jss548/miniconda3:$PATH"
    source activate radar-dev
    python {python_script} -j {cpu} -r {rid} -d1 {d1} -d2 {d2}
    """.format(time=walltime, cpu=ncpu, mem=mem, rid=radar_id, d1=radar_start, d2=radar_end, python_script=py_script, qmode=queue_mode)
    return conf_txt

#main function


#paths
qsub_path    = "/home/548/jss548/dev/tmp/qsub_dp_rain"

py_script    = '/home/548/jss548/dev/projects/PST/DPrain/retrieval/op-production.py'
run_qsub_cmd = "qsub -k oe"
ncpu         = 8
mem          = 64
walltime     = 24
start_year   = 2021
end_year     = 2021
rid_list     = [2]#[31,32,95]#,71,66]
run_flag     = True
qmode        = 'normal'
#month_list   = [[1,3],[4,6],[7,9],[10,12]] #quaterly
month_list   = [[10,10],[11,11],[12,12]]#[[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],[9,9],[10,10],[11,11],[12,12]] #one month

#S band DP test period is for Nov 2018 to March 2019
#C band DP test period is for April 2020 to May 2020


################################################
#
#####configurations######
#
# nonDoppler radars quarterly - 16 cores - 24hrs
# Doppler radars 1 month - 16 cores - 24hrs
#
#normal CPU 8, mem 32 walltime 24 #normalsl: CPU 8, mem 32, walltime 24
################################################


count = 0
for year in range(start_year, end_year+1):
    for rid in rid_list:
        
        for i, month_set in enumerate(month_list):
        
            # if i+1 not in quart_idx:
            #     continue
        
            #convert to string
            rid = str(rid).zfill(2)

            #generate qsub for wuater sets
            month_end_day = calendar.monthrange(year, month_set[1])[1]
            start_date = str(year) + str(month_set[0]).zfill(2) + '01'
            end_date   = str(year) + str(month_set[1]).zfill(2) + str(month_end_day)

            f     = gen_qsub(rid, start_date, end_date, ncpu, mem, walltime, py_script, qmode)
            fname = '/'.join([qsub_path, "dp_%s_%04i_m%i_m%i.pbs" % (rid, year, month_set[0], month_set[1])])
            with open(fname, 'w') as fid:
                fid.write(f)
            count += 1
            if run_flag == True:
                cmd           = ' '.join([run_qsub_cmd,fname])
                process       = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
                output, error = process.communicate()
                print('running ' + fname)
                print(output)

print('finished qsub generate for ',str(count), 'jobs')

running /home/548/jss548/dev/tmp/qsub_dp_rain/dp_02_2021_m10_m10.pbs
b'33897369.gadi-pbs\n'
running /home/548/jss548/dev/tmp/qsub_dp_rain/dp_02_2021_m11_m11.pbs
b'33897370.gadi-pbs\n'
running /home/548/jss548/dev/tmp/qsub_dp_rain/dp_02_2021_m12_m12.pbs
b'33897372.gadi-pbs\n'
finished qsub generate for  3 jobs
